In [7]:
import itertools
from bidict import bidict
from collections import defaultdict
import time 

WE = defaultdict(lambda: None)
WE[0] = 0 
WE[1] = 1

m=defaultdict(lambda: None)

#outputs the Wedderburn number for n
def W(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    if WE[n] == None: 
        if n % 2 == 1:
            s = sum([W(i) * W(n-i) for i in range(1, (n+1)//2)])
            WE[n] = s
        else:
            s = sum([W(i) * W(n-i) for i in range(1, n//2)])
            s += (W(n//2) * (W(n/2) + 1)) / 2  #W(n/2) choose 2 + W(n/2)
            WE[n] = s
    return WE[n]

isom_classes= defaultdict(bidict)

#The first three binary trees, and their root splits
#Note that here, and throughout, we have root split ((n_0,s_0),(n_1,s_1)) 
#where n_0 and n_1 are the numbers of leaves in the left and right tree
#s_0 and s_1 indicate which tree of this size 
#(the s_0th in the lex order and s_1st in the lex order on n_0/n_1 trees resp.)
isom_classes[1] = bidict({0 : ((1,0),(0,0))})
C_1 = (1,0)
isom_classes[2] = bidict({0 : ((1,0),(1,0))})
C_2 = (2,0)
isom_classes[3] = bidict({0 : ((2,0),(1,0))})
C_3 = (3,0)

import itertools

#in: L list, unordered with duplicates
#out: ordered list L without duplicates
def Oset(L):
    return sorted(list(dict.fromkeys(L)))

#inputs two list of pairs of integers lst1 and lst2 
#outputs a list of ordered pairs ((x,y),(z,w)) 
#such that (x,y) <= (z,w), and (x,y) in lst1 and (z,w) in lst2 or vice versa
def unordered_pairs(lst1, lst2):
    if len(lst1) == 0 or len(lst1) == 0:
        return []
    else: 
        L = [(y, x) if x <= y else (x, y) for x, y in set(itertools.product(lst1, lst2))]
        return Oset(L)

#inputs tree_0 = (n_0,s_0) and tree_1 = (n_1,s_1)
#outputs tree = (n,s) such that tree is tree_0 \oplus tree_1 
def binary_tree(tree_0,tree_1):
    ((n_0,s_0),(n_1,s_1)) = sorted((tree_0,tree_1))
    if n_0 == 0: 
        return (n_1,s_1)
    if n_1 == 0: 
        return (n_0,s_0)
    n = n_0 + n_1
    if n_0 == n_1:
        y = W(n_0)
        s = sum([ W(n - i) * W(i) for i in range(1,n_0)]) + sum([ y-i for i in range(s_0)]) + s_1 - s_0
    else: 
        s = sum([ W(n - i) * W(i) for i in range(1,n_0)]) +  W(n_0) * s_1 + s_0
    return (n,s)

# input: pairs of integers (n,s)
# output: the root split of s'th binary tree (in lex order) on n leaves in the form ((n_0,s_0),(n_1,s_1))
def root_split_finder(pair):
    n=pair[0]
    if n == 1:
        return ((1,0),(0,0))
    elif n % 2 == 1:
        t = pair[1]
        i = 1
        while W(n-i) * W(i) <= t: 
            t-=W(n-i) * W(i)
            i += 1
        (s_1,s_2) = divmod(int(t),int(W(i)))
        return ((n-i,s_1),(i,s_2))
    else:
        t = pair[1]
        i = 1
        while i <= n/2 and W(n-i) * W(i) <= t: 
            t-=W(n-i) * W(i)
            i+=1      
        (s_1,s_2) = divmod(int(t),int(W(i)))
        if i != n/2:
            return ((n-i,s_1),(i,s_2))
        else:
            y = W(n//2)
            j = 0
            r = 0
            while y-j-1 < t:
                t -= (y-j)
                j += 1
                r += 1
            return ((n//2,t+r),(n//2,j))     

#intake:  pair = (n,s) and an integer k
#output: the deck of (n,s) as an ordered set of pairs
def deck(pair, k):
    if m[(pair,k)] == None:
        n = pair[0]
        s = pair[1]
        if k > n: 
            m[(pair,k)] = []
        if k == 0: 
            m[(pair,k)] = [(0, 0)]
        if k == n:
            m[(pair,k)] = [pair]
        if n > k > 0:
            deck_set = []
            root_split = root_split_finder(pair)
            left_tree = root_split[0]
            right_tree = root_split[1]          
            r = min(k,left_tree[0])
            for i in [x for x in range(r+1) if k-x <= right_tree[0]]:              
                for element in unordered_pairs( deck( left_tree, i) , deck( right_tree,k-i) ):
                    card = binary_tree(element[0],element[1])
                    deck_set.append(card)
            m[(pair,k)] = Oset(deck_set)
    return m[(pair,k)]

#in: a binary tree in the form pair = (n,s)  
#out: draws it in ascii_art  (using the BinaryTree class of Sagemath)
def draw(pair):
    def convert_back(pair):
        T_1 = BinaryTree([])
        (n,s) = (pair[0],pair[1])
        if n == 1:
            return T_1
        if n > 1:
            pair = root_split_finder((n,s))
            T_left = convert_back(pair[0])
            T_right = convert_back(pair[1])
            return BinaryTree([T_left,T_right])
    print(ascii_art(convert_back(pair)))    

#input: k\geq 1
#output: list of universal trees for k as tuples
#(must read the file "universal_trees_k" in the same folder as this program)
def universal(k): 
    with open(f'universal_trees_{k}.txt', 'r') as file:
        lst= []
        for i, line in enumerate(file):
            lst.append(tuple(eval(line)))
        return lst  

#input: k = leaf-number, U = (n,s), a candidate for a universal tree for k-trees
#output: True iff the k-deck of U is equal to the Wedderburn number W(k),
#then write U on the text file universal_trees_k as a new line
def universal_update(T,k,extra_information = False):
    deck_size = len(deck(T,k))
    if deck_size == W(k):
        cards = [ V for V in deck(T,T[0]-1) if len(deck(V,k)) ==  W(k) ]
        if extra_information:
            print("The tree is ",k,"-universal, because the ",k,"-deck size of it is equal to Wedderburn(",k,") = ",W(k))
            print("Among its n-1-cards,", len(cards), "many of them also satisfy this.")
        with open(f'universal_trees_{k}.txt', 'a') as file:
            file.write(str(T) + "\n")
    elif deck_size != W(k) and extra_information:
        print( "The tree is NOT ",k,"-universal, because the ",k,"-deck size of this tree is", deck_size, "which is less than Wedderburn(",k,")=",W(k) )

#in: a tree = (n,s) 
#out: the tree (n+1,t) = (n,s) \oplus (1,0)
def add_one(T):
    return binary_tree(T,C_1)

#in: a tree = (n,s) 
#out: the tree (n+2,t) = (n,s) \oplus (2,0)
def add_two(T):
    return binary_tree(T,C_2)

#in: a tree = (n,s) 
#out: the tree (n+3,t) = (n,s) \oplus (3,0)
def add_three(T): 
    return binary_tree(T,C_3)


# input: k \geq 4
# output: list of universal trees on n>k vertices containing trees with k leaves
def list_Universal(k):
    n = k
    wedderburn_etherington_number = W(k)
    if W(n) != len(deck((n,0),n)) :
        U = False
    else:
        U = True
        universal_update((n,0),k)

    while U == False:
        n+=1
        for j in range(W(n)):
            deck_size = len(deck((n,j),k))
            if deck_size == wedderburn_etherington_number:
                # print((n,j))
                # draw((n,j))
                U = True
                universal_update((n,j),k)
        
       
# print("started")
# start_time = time.time()

#We define the candidate Z and specify k\geq 1
k=7

for ell in range(1,k):
    list_Universal(ell)
    
U = universal(5)[0]
# print(U)
V = universal(3)[0]
# print(V)
T = binary_tree(U,V)
# print(T)
Z = add_one(T)
draw(Z)

#We define the candidate Z and specify k\geq 1
universal_update(Z,k,extra_information = True)

# print("time passed", time.time() - start_time, "seconds")
# print("finished")

                       _____o______
                      /            \
                _____o______        o
               /            \       
           ___o____         _o_     
          /        \       /   \    
       __o___       o     o     o   
      /      \           / \        
    _o_       o         o   o       
   /   \     / \                    
  o     o   o   o                   
 / \                                
o   o                               
The tree is  7 -universal, because the  7 -deck size of it is equal to Wedderburn( 7 ) =  11
Among its n-1-cards, 0 many of them also satisfy this.
